In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

/Users/maruthienugula/RAG_learnings/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# step1 : Load and split the dataset
loader = TextLoader("langchain-crewai-dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)

In [4]:
print(len(chunks))
chunks

190


[Document(metadata={'source': 'langchain-crewai-dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and'),
 Document(metadata={'source': 'langchain-crewai-dataset.txt'}, page_content='LLMs with tools, APIs, vector databases, and other knowledge sources. (v1)'),
 Document(metadata={'source': 'langchain-crewai-dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse chains using s

In [6]:
# Model embedding and retrieving
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
vectorstore = FAISS.from_documents(chunks,embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k':8})

In [7]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1312aa390>, search_kwargs={'k': 8})

In [8]:
# step 4 : LLM and Prompt

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model("groq:llama-3.3-70b-versatile")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1312ac110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x134d7d8b0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
# Query expansion
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.
Just don't tell me you are expanding. Just expand
Original query: "{query}"

Expanded query:
""")

query_expansion_chain = query_expansion_prompt | llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.\nJust don\'t tell me you are expanding. Just expand\nOriginal query: "{query}"\n\nExpanded query:\n')
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1312ac110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x134d7d8b0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [13]:
query_expansion_chain.invoke({"query":"Langchain memory ?"})

'"Langchain memory" OR "Langchain knowledge retention" OR "memory augmentation in Langchain" OR "information recall in Langchain" OR "Langchain data storage" OR "Langchain learning capacity" OR "memory limits in Langchain" OR "optimizing Langchain memory usage" OR "Langchain memory management" OR "improving Langchain knowledge base" OR "Langchain cognitive architecture" OR "memory-based Langchain applications" OR "Langchain artificial intelligence memory" OR "large language model memory in Langchain" OR "conversational AI memory in Langchain" OR "natural language processing memory in Langchain" OR "memory-intensive Langchain tasks" OR "Langchain memory optimization techniques"'

In [ ]:
# Actually this should be done before passing into the rag_chain but not done now
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)


In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# RAG answering prompt
answer_prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

rag_chain = (
     RunnableMap({
        "input":lambda x:x["input"],
        "context": lambda x: retriever.invoke(query_expansion_chain.invoke({"query":x["input"]}))
    })
    | RunnablePassthrough.assign(
        answer = answer_prompt | llm | StrOutputParser()
    )
)
rag_chain

{
  input: RunnableLambda(...),
  context: RunnableLambda(...)
}
| RunnableAssign(mapper={
    answer: PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {input}\n')
            | ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1312ac110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x134d7d8b0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))
            | StrOutputParser()
  })

In [16]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_chain.invoke(query)
print("✅ Answer:\n", response)

{'input': 'What types of memory does LangChain support, including short-term, long-term, episodic, semantic, working, and procedural memory, as well as memory architectures such as transformer, recurrent neural network (RNN), and long short-term memory (LSTM) models, and how does it utilize memory-augmented neural networks, external knowledge graphs, and retrieval-based approaches to enhance language understanding and generation capabilities?'}
✅ Answer:
 {'input': 'What types of memory does LangChain support?', 'context': [Document(id='7e08b0dc-ef5b-45a0-b82d-8154c715510f', metadata={'source': 'langchain-crewai-dataset.txt'}, page_content='LangChain also supports hybrid retrieval, which combines keyword-based (sparse) retrieval methods like BM25 with embedding-based (dense) retrieval. This ensures better recall by catching both exact term matches and semantically similar content. (v8)'), Document(id='8697a729-d7e4-43f8-9a5d-029d47d619a0', metadata={'source': 'langchain-crewai-dataset.

In [17]:
response = rag_chain.invoke(query)
print("✅ Answer:\n")
response

✅ Answer:



{'input': 'What types of memory does LangChain support?',
 'context': [Document(id='ebffc844-fb8b-49d3-8a9f-4201dc6fcdd1', metadata={'source': 'langchain-crewai-dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v6)'),
  Document(id='406b904d-f65c-4e72-b5b3-e6a4cfeb706a', metadata={'source': 'langchain-crewai-dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v8)'),
  Document(id='f2ba97b4-3e47-43e8-b10e-333e5156b242', metadata={'source': 'langchain-crewai-dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These all

In [18]:
response["answer"]

'LangChain supports two types of memory modules: \n\n1. ConversationBufferMemory: This allows the LLM to maintain awareness of previous conversation turns.\n2. ConversationSummaryMemory: This allows the LLM to summarize long interactions to fit within token limits.'